In [1]:
# Data Loader
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from pytorch_lightning.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import json
#from CustomDataset import * 
from Horizon_DataLoader import * 
import sys
sys.path.append('../')
from config import *
from file_helper import *
#from Horizon_and_SAM.Horizon import PE_helper
from  PE_helper import *

from pytorch_lightning.callbacks import ModelCheckpoint , Callback

def collate_fn(batch):
    return tuple(zip(*batch))
#=================================
#             Augmentation
#=================================

class CustomDataModule(pl.LightningDataModule):
    def __init__(self ,
                 train_dir ,
                 test_dir , batch_size = 2,
                 num_workers = 0 , img_size=[IMG_WIDTH, IMG_HEIGHT] , use_aug = True ,padding_count = 24 ,c =0.1
                   ):
        super().__init__()
        self.train_dir = train_dir
        self.test_dir = test_dir
        
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.img_size = img_size      
        self.use_aug = use_aug
        self.padding_count  = padding_count
        self.c = c
        

        pass

    def prepare_data(self) -> None:
        # Download dataset
        pass

    def setup(self, stage):
        # Create dataset...          
                
        self.entire_dataset = CustomDataset(self.train_dir  , use_aug= self.use_aug ,  c=self.c , img_size=self.img_size)
        self.train_ds , self.val_ds = random_split(self.entire_dataset , [0.9, 0.1])        
        self.test_ds = CustomDataset(self.test_dir  , use_aug= False , img_size=self.img_size  )
        
        print("image size ",self.img_size)
        pass

    # ToDo: Reture Dataloader...
    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(self.train_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=True)
    
    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.val_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=False)
    
    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=True)

    pass


# Test
dm = CustomDataModule ( train_dir= f"../anno/test_visiable_10_no_cross.json" ,
                       test_dir= f"../anno/test_visiable_10_no_cross.json" , padding_count=1024, img_size=[1024,512]
                       )


d:\conda\envs\layout\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\Projects\Layout\NTHU_CGV_Layout_exp\Horizon_and_SAM\Horizon
d:\Projects\Layout\NTHU_CGV_Layout_exp\Horizon_and_SAM\Horizon


In [2]:
from scipy.ndimage.filters import maximum_filter
def find_N_peaks(signal, r=29, min_v=0.05, N=None):
    max_v = maximum_filter(signal, size=r, mode='wrap')
    pk_loc = np.where(max_v == signal)[0]
    pk_loc = pk_loc[signal[pk_loc] > min_v]
    if N is not None:
        order = np.argsort(-signal[pk_loc])
        pk_loc = pk_loc[order[:N]]
        pk_loc = pk_loc[np.argsort(pk_loc)]
    return pk_loc, signal[pk_loc]


a = np.random.randn(1024)
pk_loc , signal = find_N_peaks(a )
print(pk_loc.shape)
print(pk_loc)
print(signal)

(37,)
[  16   33   55   83  109  138  172  193  235  254  303  327  347  386
  428  458  475  513  540  562  579  608  638  670  690  715  739  756
  781  797  814  854  888  934  949  979 1013]
[2.51659653 1.12392401 2.09382046 1.95041126 1.8413406  1.77542224
 2.04639904 1.56116418 1.79070556 1.82544007 1.6479888  1.43265075
 0.96122704 2.61811188 2.14218705 2.50768573 1.14721191 2.13744046
 2.03460155 1.62007709 1.8330931  2.00848849 1.93099906 1.66113793
 1.85647599 1.67839423 1.51555779 2.59150762 1.89029799 1.57418452
 1.74909214 1.72317446 2.21240185 1.85285085 2.08521843 2.59867129
 2.85340091]


In [22]:
from raw_model_pixelwize_bbox import *
from Horizon_DataLoader import  * 
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance

from torch import Tensor
def unpad_data( x :[Tensor] ) :
	non_zero_indices = torch.nonzero(x)
	#print(non_zero_indices)
	# Get the non-zero values
	non_zero_values = x[non_zero_indices[:,0], non_zero_indices[:,1]]

	unique = torch.unique(non_zero_indices[:,0] ,return_counts=True)	
	non_zero_values = torch.split(non_zero_values , tuple(unique[1]))
	
	return non_zero_values

# ToDo: fix this
def match_gt(gt_u_b , gt_vtop_b , gt_vbtm_b , gt_du_b , gt_dvtop_b , gt_dv_btm_b , out_box , out_cls ,input_b['u_grad']  ):
    gt_box =  torch.vstack([ u, vtop,vbtm,  du ,dvtop , dvbtm]).permute(1,0)   # [n , 6]  

    pass

class Model(pl.LightningModule):    
    def __init__(self):
        super().__init__()
        self.model = HorizonNet(backbone='resnet50', use_rnn=True)
        
    def forward(self , x):
        prob , boxs  = self.model(x)  # [ b , _ , 1024]
        prob = prob.permute((0,2,1))
        boxs = boxs.permute((0,2,1))
        print("prob" , prob.shape) # [ b , 1024 , 1]
        print("boxs" , boxs.shape) # [ b , 1024 , 5]

        return prob , boxs
        pass

    def training_step(self , input_b ,batch_idx ):
        img = input_b['image']        
        gt_pro = input_b['u_grad']

        gt_u_b = unpad_data( input_b['u'])          
        gt_vtop_b =unpad_data(input_b['v_top'])
        gt_vbtm_b = unpad_data (input_b['v_btm'])
        gt_du_b = unpad_data(input_b['du'])
        gt_dvtop_b = unpad_data(input_b['dv_top'])
        gt_dv_btm_b = unpad_data(input_b['dv_btm'])

        gt_boxes = []
        for u,vtop,vbtm,du,dvtop, dvbtm in zip(gt_u_b , gt_vtop_b , gt_vbtm_b , gt_du_b , gt_dvtop_b , gt_dv_btm_b  ):
            gt_box =  torch.vstack([ u, vtop,vbtm,  du ,dvtop , dvbtm]).permute(1,0)   # [n , 6]  
          

        out_prob , out_boxs = self.forward(img)
        
        # ToDo: Encode GT
        gt_counts = [a.shape[0] for a in gt_u_b]
        print("gt_counts" , gt_counts)
        matched_idxs = []
        matched_boxs = []
        for pred_prob , pred_box , gt_cnt , gt_u in zip( out_prob.detach().cpu().numpy().astype(np.float32) , out_boxs , gt_counts , gt_u_b):
            # gt_box =  torch.vstack([ u, vtop,vbtm,  du ,dvtop , dvbtm]).permute(1,0)   # [n , 6]      
            print("pred_prob" , pred_prob.flatten().shape)
            pkloc , signal = find_N_peaks( pred_prob.flatten() , min_v=-1, N=gt_cnt)            
            print("pkloc" , pkloc)
            print("pkloc" , pkloc.shape)
            # turn index to u ,  find the cloest u
            pred_u = pkloc / 1024
            print("gt_u", gt_u.shape)
            u_cdist =distance.cdist( np.expand_dims(pred_u, axis=0),  gt_u.unsqueeze(-1).detach().cpu().numpy() )
            row_idx , col_idx = linear_sum_assignment ( u_cdist )
            print("row_idx" , row_idx)
            print("col_idx" , col_idx)
            matched_idxs.append(pkloc)
            #matched_boxs.append()
            l1_loss = F.l1_loss(pred_box[row_idx] , gt[col_idx]   )

        


        # ToDo: calculate loss
        cls_loss = F.binary_cross_entropy_with_logits(out_prob , gt_prob)
        box_loss = F.l1_loss(out_boxs , encode_gt )
        total_loss = cls_loss + box_loss
        return total_loss
    
    def validation_step(self, input_b, batch_idx):
        #ToDo...
        pass
    
    def configure_optimizers(self):
        #ToDo: Look at paper 
        opt = optim.Adam(self.parameters() , lr=0.00035)
        # Ref: https://github.com/sunset1995/HorizonNet/blob/master/train.py#L44

        return [opt] , []

dm = CustomDataModule ( train_dir= f"../../anno/train_visiable_20_no_cross.json" ,
                        test_dir= f"../../anno/test_visiable_10_no_cross.json" ,
                        #test_dir= f"../anno/train_visiable_20_no_cross.json" ,
                        padding_count=1024,
                        use_aug=False , c= 0.65,batch_size=1,
                        img_size=[1024,512]
                       )


trainer = pl.Trainer(accelerator='gpu' , devices=1 ,
                    min_epochs=1, max_epochs=20 , precision=16 , fast_dev_run=True  ,
                    callbacks=[])
m=Model()
trainer.fit(m , dm)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Epoch 0:   0%|          | 0/2 [00:35<?, ?it/s]


You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


image size  [1024, 512]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | HorizonNet | 81.6 M
-------------------------------------
81.6 M    Trainable params
0         Non-trainable params
81.6 M    Total params
163.165   Total estimated model params size (MB)
d:\conda\envs\layout\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
d:\conda\envs\layout\lib\site-packages\pytorch_lightning\trainer\trainer.py:1613: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the traini

Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] prob torch.Size([1, 1024, 1])
boxs torch.Size([1, 1024, 5])
gt_counts [1]
pred_prob (1024,)
pkloc [906]
pkloc (1,)
gt_u torch.Size([1])
row_idx [0]
col_idx [0]


TypeError: append() takes exactly one argument (0 given)